In [23]:
library(keras)
library(dplyr) 
use_condaenv('r-tensorflow')

The function get.scale returns a list with two parameters : first the mean, second the std. If the input is already centered (it has an attribute center which is taken up by attr), we extract the mean. If it's not, we put the mean by default at 0. If the input is already scale (it has an attribute scale), we extract the std, if it's not we put the std by default at 1.
The function scale.as enables to scale a data set X2 the same way X1 was scaled

In [64]:
get.scale <- function(scaled) {
  if ("scaled:center" %in% names(attributes(scaled))) {
      center <- attr(scaled, "scaled:center")
  } else {
      center <- rep(0, ncol(scaled))
  }
  if ("scaled:scale" %in% names(attributes(scaled))) {
      list(center, attr(scaled, "scaled:scale"))
  } else {
      list(center, rep(1., length(center)))
  }
}
scale.as.x <- function(x, scaled) {
  s <- get.scale(scaled)
  centered <- sweep(x, 2, s[[1]])
  sweep(centered, 2, s[[2]], FUN = "/")
}

scale.as.y <- function(y, scaled) {
    s <- get.scale(scaled)
    (y - s[[1]])/s[[2]]
}

unscale.y <- function(y, scaled) {
    s <- get.scale(scaled)
    y * s[[2]] + s[[1]]
}

In [65]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))


In [66]:
#searching for input variables with zero variance (without the varible Intensity)
x = train.data[, -c(2,3)]
y = train.data$VALENCE.PLEASANTNESS
x$Intensity <- as.numeric(train.data$Intensity)
idx.zero.var <- apply(x, 2, var) == 0
#x is the whole training data (only predictors)
x <- x[,!idx.zero.var]

full.data = data.frame(x,y)

In [67]:
#Split the data between a training and a test set
set.seed(100)
len <- length(x[,1])
idx.train <- sample(1:len, 2*len/3)

train.x <- x[idx.train,]
train.y <- y[idx.train]
test.x <- x[-idx.train,]
test.y <- y[-idx.train]

train.x.prep <- scale(train.x, center = T, scale = T)
train.y.prep <- scale(train.y, center = T, scale = T)

test.x.scaled = scale.as.x(test.x,train.x.prep)
test.y.scaled = scale.as.y(test.y,train.y.prep)
#We have 472 rows in the training set, and 472 for the test set
#Training data is scaled 

In [76]:
nn <- keras_model_sequential()
nn <- nn %>%
      layer_dense(units = 100, activation = 'relu', input_shape = c(dim(train.x)[2]), 
                  kernel_regularizer = regularizer_l2(l = 0.001)) %>%
      #layer_dropout(rate=0.25)%>%
      layer_dense(units = 100, activation = 'relu') %>%
      #layer_dropout(rate=0.25)%>%
      layer_dense(units = 100, activation = 'relu') %>%
      layer_dense(units = 1, activation = 'linear')

In [77]:
nn %>% compile(optimizer = "adam", loss = "mean_squared_error", metrics = list("mean_absolute_error"))

In [78]:
history <- nn %>% fit(as.matrix(train.x.prep),
                      as.matrix(train.y.prep),
                      verbose = 0,
                      batch_size = 100,
                      validation_data = list(as.matrix(test.x.scaled),as.matrix(test.y.scaled)),
                      epochs = 500)

In [80]:
nn.pred <- predict(nn, as.matrix(test.x.scaled))
nn.pred.unscaled = unscale.y(nn.pred, train.y.prep)

sqrt(mean((nn.pred.unscaled - test.y)^2))

[1] 23.57819